# VGG16을 사용하여 아이돌 이미지 분류
- 이미지는 Kaggle에서 다운로드 가능(https://www.kaggle.com/datasets/vkehfdl1/kidf-kpop-idol-dataset-female?resource=download&select=kid_f_train.csv)


In [1]:
import numpy as np
import pandas as pd
import os

from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

c:\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url = './dataset'

csv_url_train = url + '/kid_f_train.csv'
csv_url_test = url + '/kid_f_test.csv'


train_labels = pd.read_csv(csv_url_train)
test_labels = pd.read_csv(csv_url_test)

In [3]:
# 라벨값 비율 확인
print('훈련 데이터셋 shape > ', train_labels.shape)
print('훈련 데이터셋 비율 top 5')
print((train_labels.groupby('name').size() / train_labels.shape[0] * 100).sort_values(ascending=False).head(5))

print('\n테스트 데이터셋 shape > ', test_labels.shape)
print('테스트 데이터셋 비율 top 5')
print((test_labels.groupby('name').size() / test_labels.shape[0] * 100).sort_values(ascending=False).head(5))

훈련 데이터셋 shape >  (5591, 2)
훈련 데이터셋 비율 top 5
name
lisa        22.929708
rose        22.267931
jisoo       18.297263
jennie       7.404758
kimminju     4.668217
dtype: float64

테스트 데이터셋 shape >  (300, 2)
테스트 데이터셋 비율 top 5
name
jisoo     24.000000
iu        21.333333
rose      14.000000
lisa      13.333333
jennie     5.000000
dtype: float64


# 데이터 전처리
- 훈련 데이터셋 중 lisa, rose, jisoo의 비율이 절반 이상이 넘음 => 따라서 세 명은 훈련 데이터, 학습 데이터에서 삭제
- 텐서화 
 - 세명의 데이터를 삭제한 데이프레임을 기준으로 이미지 불러와서 텐서화
 - 이미지파일만 있는 것이 아니라 
 Thumbs.db' 파일도 있으므로 텐서로 변활할 때 에러 발생이 예상되므로 예외처리 하기


In [4]:
# 리사, 로제, 지수 제거
drop_name = ['lisa', 'rose', 'jisoo']

train_drop_index = train_labels[train_labels['name'].isin(drop_name)].index
test_drop_index = test_labels[test_labels['name'].isin(drop_name)].index

train_labels = train_labels.drop(train_drop_index)
test_labels = test_labels.drop(test_drop_index)

In [5]:
print("###### 세 명 삭제 후 ######")
print("train_labels 데이터 개수 : ", len(train_labels))
print("train_labels 클래스 개수 : ", len(train_labels['name'].unique()))
print('----------' * 3)
print("test_labels 데이터 개수 : ", len(test_labels))
print("test_labels클래스 개수 : ", len(test_labels['name'].unique()))

###### 세 명 삭제 후 ######
train_labels 데이터 개수 :  2041
train_labels 클래스 개수 :  52
------------------------------
test_labels 데이터 개수 :  146
test_labels클래스 개수 :  22


In [6]:
def image_to_tensor(df, dir_url):
    file_list = os.listdir(dir_url)
    result_list = []

    for i in tqdm(range(df.shape[0])):
        try:
            file_name = df['file_name'].iloc[i] # 파일 이름 Ex)100.jpg
            label = df['name'].iloc[i] # 연예인 이름

            # 이미지 오픈 및 텐서화
            file_url = dir_url + '/' + file_name
            img = Image.open(file_url)
            tf = transforms.ToTensor()
            tensor_img = tf(img)
            # 튜플 형태로 저장
            result_list.append((tensor_img, label))
        except:
            print("파일명 > ", file_name, "\n연예인명 > ", label)
            
    return result_list

In [7]:
train_dir_url = './dataset/HQ_512x512/HQ_512x512' 
test_dir_url = './dataset/test_final_with_degrad/test'

train_dataset = image_to_tensor(train_labels, train_dir_url)
test_dataset = image_to_tensor(test_labels, test_dir_url)

 67%|██████▋   | 98/146 [00:00<00:00, 121.37it/s]

파일명 >  1792 .jpg 
연예인명 >  joyuri


100%|██████████| 146/146 [00:01<00:00, 83.00it/s]


In [8]:
print("train_dataset 데이터 개수 : ", len(train_dataset))

lst = []
for i in range(len(train_dataset)):
    lst.append(train_dataset[i][1])
s_lst = pd.Series(lst)
print('train_dataset 라벨 개수', len(s_lst.unique()))

print('----------' * 3)

print("test_dataset 데이터 개수 : ", len(test_dataset))

lst = []
for i in range(len(test_dataset)):
    lst.append(test_dataset[i][1])
s_lst = pd.Series(lst)
print('test_dataset 라벨 개수', len(s_lst.unique()))

train_dataset 데이터 개수 :  2041
train_dataset 라벨 개수 52
------------------------------
test_dataset 데이터 개수 :  145
test_dataset 라벨 개수 22


# 모델링

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [10]:
# DataLoader 만들기
batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_laoder = DataLoader(dataset=test_dataset, batch_size=batch_size)

In [15]:
# VGG 모델은 여러가지 타입이 있으므로 타입에 맞춰 모델 생성하기 위해 layer를 거친 후 출력 kernel의 개수 미리 지정(M은 pooling layer)
VGG_types = {
    'VGG11' : [64, 'M', 128, 'M', 256,256, 'M', 512,512, 'M',512,512,'M'],
    'VGG13' : [64,64, 'M', 128, 128, 'M', 256, 256, 'M', 512,512, 'M', 512,512,'M'],
    'VGG16' : [64,64, 'M', 128, 128, 'M', 256, 256,256, 'M', 512,512,512, 'M',512,512,512,'M'],
    'VGG19' : [64,64, 'M', 128, 128, 'M', 256, 256,256,256, 'M', 512,512,512,512, 'M',512,512,512,512,'M']
}

def make_layer(config): # 위에서 정의한 타입별로 모델 생성
    layers = []
    in_planes = 3 # input 개수
    for value in config: 
        if value == "M": # Pooling layer 일 때
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        else: # Conv layer 일 때
            layers.append(nn.Conv2d(in_planes, value, kernel_size=3, padding=1)) 
            layers.append(nn.ReLU())
            in_planes = value
    return nn.Sequential(*layers)

In [16]:
class VGG(nn.Module):
    def __init(self, config, num_class=52): # model : 만들 VGG 모델 형태 
        super(VGG, self).__init__()
        self.features = make_layer(config)
        
        # Fully Connected Layer 쌓기
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, num_class)
        )
    
    # 학습
    def forward(self, x):
        out = self.features(x)
        out = torch.flatten(out,1)
        out = self.classifier(out)
        return out
        

In [17]:
def VGG11():
    return VGG(config=VGG_types['VGG11'])

def VGG13():
    return VGG(config=VGG_types['VGG13'])

def VGG16():
    return VGG(config=VGG_types['VGG16'])

def VGG19():
    return VGG(config=VGG_types['VGG19'])

In [ ]:
model = VGG11()